# Score matching examples

Moved testing and plotting here as the main functions seem stable-ish

In [23]:
using NBInclude
nbinclude("ScoreMatching_functions.ipynb")

coordinate_descent (generic function with 1 method)

## Simulate from the model, then try to fit it

In [27]:
# Simulate with dimensions ex_dims = [M, D, F, N]. b is assumed given and known, ω=1/T=1

ex_dims =[5, 2, 4, 3]
ex_params = create_rand_params(ex_dims...);
ex_params.b = log(5)*ones(size(ex_params.b));
out = sample_spikes(getall(ex_params)...);

j_true = J(out, getall(ex_params))

#rand_params = deepcopy(ex_params);
#rand_params.ϕ = 2*π*rand(size(rand_params.ϕ))
rand_params = create_rand_params(ex_dims...);

j_rand = J(out, getall(rand_params))

@show j_true[1], j_rand[1]

@show size(out[1][1])
@show size(j_true[2][1,1])

par = getall(rand_params);


(N,M,D) = (3,5,2)
(j_true[1],j_rand[1]) = (-458.23958410146605,1002.6395321812965)
size((out[1])[1]) = (3,)
size((j_true[2])[1,1]) = (4,3)


In [28]:
@show size(out)
@show size(out[1])
@show size(out[1][1])

par[3]

size(out) = (3,)
size(out[1]) = (5,)
size((out[1])[1]) = (3,)


4x3 Array{Float64,2}:
 4.54945  3.41623   0.150635
 4.59755  3.40669   4.59999 
 2.12151  0.232736  4.72249 
 2.1107   1.07387   2.72805 

In [37]:
# Run coordinate descent
par = getall(rand_params);

penalise_integrals = Array(Any, 3)
penalise_integrals[2] = 1e1;
penalise_integrals[3] = [];

optim_params = Dict()
optim_params[:iterations] = 10

par, S = coordinate_descent(out, par, max_iter=5, penalise_integrals=penalise_integrals, optim_params=optim_params);
#par, S = coordinate_descent(out, par, max_iter=5);

(N,M,D,F) = (3,5,2,4)
Initial cost is 11250.260891719812
After setting b, cost is 1039.8598332039312


Starting iteration 1...............
Updated θ ----------------- Current cost is 31.004706070064284
Updated C ----------------- Current cost is -737.2179537253312
Updated ϕ_1 ------------ Current cost is -875.9249759045326
Updated ϕ_2 ------------ Current cost is -1074.4457528766359
Updated ϕ_3 ------------ Current cost is -1215.79023354478
Updated suff stats --------- Current cost is -1215.7902335447798


Starting iteration 2...............
Updated θ ----------------- Current cost is -1684.1325281498198
Updated C ----------------- Current cost is -1851.2311799827417
Updated ϕ_1 ------------ Current cost is -1992.7123484152473
Updated ϕ_2 ------------ Current cost is -2163.413603356765
Updated ϕ_3 ------------ Current cost is -2197.9746373193148
Updated suff stats --------- Current cost is -2197.9746373193143


Starting iteration 3...............
Updated θ ----------------- Current cos

In [38]:
using JLD
save("Example_small_with_constraint.jld", "par", par, "S", S, "data", out, "rand_params", rand_params, "ex_params", ex_params)

In [47]:
using JLD
#tmp = load("Example_large.jld")
tmp = load("Example_small_no_constraint.jld")
tmp = load("Example_small_with_constraint.jld")
par = tmp["par"]
ex_params = tmp["ex_params"]
rand_params = tmp["rand_params"]
out = tmp["data"];
C, θ, ϕ, b, ω, N, M, D, F = name_params(par);
@show N,M,D,F


(N,M,D,F) = (3,5,2,4)


(3,5,2,4)

In [48]:
# Show number of spikes, integral of true rate function and integral of estimated rate function
for j1 = 1:ex_dims[1] #neur
    for i1=1:ex_dims[4] #trial
    bla = lambda_functions(0:0.01:ex_params.T, ex_params.C, ex_params.θ; b=ex_params.b, phi=ex_params.ϕ[:,i1], omega=ex_params.ω)
    bla_est = lambda_functions(0:0.01:ex_params.T, par[1], par[2]; b=par[4], phi=par[3][:,i1], omega=par[5])

        a= [i1 j1 size(out[i1][j1]) (sum(bla[j1,:])/length(bla[j1,:])) (sum(bla_est[j1,:])/length(bla_est[j1,:]))]
    @show a
    end
end

a = Any[1 1 (3,) 5.332178984115386 6.591532925298877]
a = Any[2 1 (7,) 5.359510816957971 7.004675347078006]
a = Any[3 1 (3,) 5.350041314991888 6.8186980405016415]
a = Any[1 2 (6,) 5.015965002254434 7.490391523406442]
a = Any[2 2 (6,) 5.014212485964839 7.572996711720233]
a = Any[3 2 (6,) 5.010034799003201 7.5891585995457]
a = Any[1 3 (1,) 5.269741341679863 7.213510253703834]
a = Any[2 3 (5,) 5.273601039292122 6.981657366945274]
a = Any[3 3 (5,) 5.243496181836071 7.013355393915764]
a = Any[1 4 (6,) 6.288791749089977 5.063612760861279]
a = Any[2 4 (4,) 6.358852293530145 5.037843500977961]
a = Any[3 4 (4,) 6.380304665764962 5.077611673774215]
a = Any[1 5 (3,) 5.177434027483218 7.979125624413505]
a = Any[2 5 (4,) 5.185272585539358 7.628144255212001]
a = Any[3 5 (4,) 5.192916220942187 7.716522552885131]


In [49]:
# Plot data, true rate function and estimated rate function for a neuron on given trials
using PlotlyJS

num_neur = 3
trials = 1:3
dim_latent = 2

#Plot estimated
to_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_rand_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
latent_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
latent_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
colors = ["rgb(0,0,255)", "rgb(0,255,0)", "rgb(255,0,0)", "rgb(128,128,0)", "rgb(0,128,128)"]
for i1 = trials
    # Plot data
    to_plot[i1] = scatter(;x=collect(out[i1][num_neur]), y=i1*collect(ones(size(out[i1][num_neur]))), mode="markers", marker_color=colors[i1], yaxis="y2")
    
    # Compute underlying rate and latent functions
    rate = lambda_functions(0:0.01:ex_params.T, ex_params.C, ex_params.θ; b=ex_params.b, phi=ex_params.ϕ[:,i1], omega=ex_params.ω)
    rate_est = lambda_functions(0:0.01:ex_params.T, par[1], par[2]; b=par[4], phi=par[3][:,i1], omega=par[5])
    rate_rand = lambda_functions(0:0.01:ex_params.T, rand_params.C, rand_params.θ; b=rand_params.b, phi=rand_params.ϕ[:,i1], omega=rand_params.ω)
    latent = latent_functions(0:0.01:ex_params.T, ex_params.θ; phi=ex_params.ϕ[:,i1], omega=ex_params.ω)
    latent_est = latent_functions(0:0.01:ex_params.T, par[2]; phi=par[3][:,i1], omega=par[5])
    
    # Plot underlying functions
    rate_plot[i1] = scatter(; 
        x=collect(0:0.01:ex_params.T), 
    y=collect(rate[num_neur,:]),
    line_color = colors[i1],
    yaxis = "y1"
)
    rate_est_plot[i1] = scatter(; 
        x=collect(0:0.01:ex_params.T), 
    y=collect(rate_est[num_neur,:]),
    line_color = colors[i1],
    yaxis = "y0"
)

    rate_rand_plot[i1] = scatter(; 
        x=collect(0:0.01:ex_params.T), 
    y=collect(rate_rand[num_neur,:]),
    line_color = colors[i1],
    yaxis = "y0"
)

    
    latent_plot[i1] = scatter(; 
        x=collect(0:0.01:ex_params.T), 
    y=collect(latent[dim_latent,:]),
    line_color = colors[i1],
    yaxis = "y1"
)
    latent_est_plot[i1] = scatter(; 
        x=collect(0:0.01:ex_params.T), 
    y=collect(latent_est[dim_latent,:]),
    line_color = colors[i1],
    yaxis = "y0"
)

end

lo = Layout(;xaxis_range=[0,1])

#plt = plot([Plot(to_plot[trials]); Plot(rate_plot[trials]);  Plot(rate_est_plot[trials]);  Plot(latent_plot[trials]);  Plot(latent_est_plot[trials])])
plt = plot([Plot(to_plot[trials], lo); Plot(rate_plot[trials], lo);  Plot(rate_est_plot[trials], lo); Plot(rate_rand_plot[trials], lo)])

relayout!(plt, height=700)

plt

In [ ]:
# Compute integrals of rate functions




In [ ]:
@time for num_neur = 1:ex_dims[1]
    for num_trial = 1:ex_dims[4]
        rate(x) = lambda_functions(x, ex_params.C, ex_params.θ; b=ex_params.b, phi=ex_params.ϕ[:,num_trial], omega=ex_params.ω)[num_neur]
        int_rate, err = hquadrature(rate, 0.0, ex_params.T; reltol=1e-8, abstol=0, maxevals=0)

        @show (size(out[num_trial][num_neur]), int_rate)
    end
end


In [ ]:
# Show the total power that gets mapped into latent space (left - true, middle - estimated, right - random starting point)
u1,s1,v1 = svd(ex_params.θ);
u2,s2,v2 = svd(par[2]);
u3,s3,v3 = svd(rand_params.θ);
[sum(v1.^2,2) sum(v2.^2,2) sum(v3.^2,2)]

In [ ]:
# Show the error in total power that gets mapped to individual neurons per frequency
sum(((ex_params.C * ex_params.θ) .- (par[1] * par[2])).^2,1)'